In [1]:
data_folder = './reuters21578/'

sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

# Word2Vec number of features
num_features = 500
# Limit each newsline to a fixed number of words
# document_max_num_words = 100
# Selected categories
# selected_categories = ['pl_usa']

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [16]:
# Create category dataframe

# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
#     print(i[1])
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)

# print "category_data: ", category_data
#(news_categories.values).tolist()

In [4]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to /home/prabha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# print(category_data)
print(category_dictionary)

{'Topics': ['acq', 'alum', 'austdlr', 'austral', 'barley', 'bfr', 'bop', 'can', 'carcass', 'castor-meal', 'castor-oil', 'castorseed', 'citruspulp', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'corn-oil', 'cornglutenfeed', 'cotton', 'cotton-meal', 'cotton-oil', 'cottonseed', 'cpi', 'cpu', 'crude', 'cruzado', 'dfl', 'dkr', 'dlr', 'dmk', 'drachma', 'earn', 'escudo', 'f-cattle', 'ffr', 'fishmeal', 'flaxseed', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-meal', 'groundnut-oil', 'heat', 'hk', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'inventories', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-meal', 'lin-oil', 'linseed', 'lit', 'livestock', 'lumber', 'lupin', 'meal-feed', 'mexpeso', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-meal', 'palm-oil', 'palmkernel', 'peseta', 'pet-chem', 'platinum', 'plywood', 'pork-belly', 'potato', 'p

In [5]:
def to_category_vector(categories):
    vector = zeros(len(categories)).astype(float32)
    
    for i in range(len(categories)):
        if target_categories[i] in categories:
            vector[i] = 1.0
    
    return vector

In [6]:
lemmatizer = WordNetLemmatizer()
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
stop_words = set(stopwords.words("english"))

def cleanUpSentence(r, stop_words = None):
    r = r.lower().replace("<br />", " ")
    r = re.sub(strip_special_chars, "", r.lower())
    if stop_words is not None:
        words = word_tokenize(r)
        filtered_sentence = []
        for w in words:
            w = lemmatizer.lemmatize(w)
            if w not in stop_words:
                filtered_sentence.append(w)
        return " ".join(filtered_sentence)
    else:
        return r

In [19]:
# Parse SGML files
document_X = {}
document_Y = {}
docid_traintest = {}
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)

# Iterate all files
for i in range(sgml_number_of_files):
    if i < 10:
        seq = '00' + str(i)
    else:
        seq = '0' + str(i)
        
    file_name = sgml_file_name_template.replace('NNN', seq)
    print('Reading file: %s' % file_name)
    #data_folder + file_name
    with open(data_folder+file_name, 'rb') as file:
        content = BeautifulSoup(file.read().lower(),'html.parser')

        for newsline in content('reuters'):
            document_categories = []

            # News-line Id
            document_id = newsline['newid']
        #             print document_id,
            train_test = newsline['lewissplit']
            docid_traintest[document_id] = train_test
        #             print "train_test: ",train_test

            # News-line text
            document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
            doc_categories=strip_tags(str(newsline('topics')[0].body))
            doc_categories = unescape(doc_categories)

            document_body = unescape(document_body)

            # News-line categories
            topics = newsline.topics.contents
            places = newsline.places.contents
            people = newsline.people.contents
            orgs = newsline.orgs.contents
            exchanges = newsline.exchanges.contents

            for topic in topics:
                document_categories.append('to_' + strip_tags(str(topic)))

            for place in places:
                document_categories.append('pl_' + strip_tags(str(place)))

            for person in people:
                document_categories.append('pe_' + strip_tags(str(person)))

            for org in orgs:
                document_categories.append('or_' + strip_tags(str(org)))

            for exchange in exchanges:
                document_categories.append('ex_' + strip_tags(str(exchange)))

            document_X[document_id] = document_body
            document_Y[document_id] = document_categories
            

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm


In [21]:
# data preprocessing
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc, stop_words))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package punkt to /home/prabha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/prabha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
totalX,encoded_docs=create_x_matrix(document_X)

In [28]:
print(len(totalX))

21000


In [52]:
words_in_body={}

for i in range(len(totalX)):
    words=totalX[i].split(' ')
    words_in_body[i]=words    

one_hot_label=[]
for key,v in words_in_body.items():
    dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
    for i in v:
        if i in category_dictionary['Topics']:
            dict_temp['Topics']+=1
        if i in category_dictionary['Places']:
            dict_temp['Places']+=1
        if i in category_dictionary['People']:
            dict_temp['People']+=1
        if i in category_dictionary['Exchanges']:
            dict_temp['Exchanges']+=1
        if i in category_dictionary['Organizations']:
            dict_temp['Organizations']+=1
            
    one_hot_label.append(dict_temp)
    
# print(one_hot_label)
# ranking=[]
# for i in one_hot_label:
#     ranking.append(list(i.values()))
# #print(np.array(ranking).shape)


ranking=[]
one_hot_label_list = []
for i in one_hot_label:
    sorted_x = sorted(i.items(), key=lambda kv: kv[1], reverse=True) 
    ranking.append(sorted_x)
    one_hot_label_list.append(list(i.values()))
print ("ranking",ranking[0])


ranking [('Topics', 7), ('Places', 2), ('People', 0), ('Exchanges', 0), ('Organizations', 0)]


In [57]:
print(np.array(ranking).shape)


(21000, 5, 2)


In [58]:
from keras.models import Sequential
from keras.layers import Dense,Flatten, Dropout,Embedding
nn = Sequential()
max_vocab_size = 200
nn.add(Embedding(200, 20, input_length=max_vocab_size))
nn.add(Dense(10, activation="relu", input_shape=(max_vocab_size,)))
nn.add(Dropout(0.15))
nn.add(Flatten())
nn.add(Dense(5,activation="softmax"))
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(np.array(encoded_docs), np.array(one_hot_label_list), batch_size=16, epochs=5,
          verbose=1, validation_split=0.2)

Train on 16800 samples, validate on 4200 samples
Epoch 1/5
16800/16800 [==============================] - 6s 329us/step - loss: 2.5853 - acc: 0.8052 - val_loss: 2.5973 - val_acc: 0.6407
Epoch 2/5
16800/16800 [==============================] - 5s 279us/step - loss: 2.3091 - acc: 0.8128 - val_loss: 2.5152 - val_acc: 0.7943
Epoch 3/5
16800/16800 [==============================] - 5s 280us/step - loss: 2.2513 - acc: 0.8118 - val_loss: 2.5070 - val_acc: 0.8021
Epoch 4/5
16800/16800 [==============================] - 5s 279us/step - loss: 2.2272 - acc: 0.8176 - val_loss: 2.4964 - val_acc: 0.8062
Epoch 5/5
16800/16800 [==============================] - 5s 280us/step - loss: 2.1993 - acc: 0.8161 - val_loss: 2.5209 - val_acc: 0.8107


In [59]:
# Iterate all files
document_X={}
document_Y={}
for i in range(21,22):
    if i < 10:
        seq = '00' + str(i)
    else:
        seq = '0' + str(i)
        
    file_name = sgml_file_name_template.replace('NNN', seq)
    print('Reading file: %s' % file_name)
    #data_folder + file_name
    with open(data_folder+file_name, 'rb') as file:
        content = BeautifulSoup(file.read().lower(),'html.parser')

        for newsline in content('reuters'):
            document_categories = []

            # News-line Id
            document_id = newsline['newid']
        #             print document_id,
            train_test = newsline['lewissplit']
            docid_traintest[document_id] = train_test
        #             print "train_test: ",train_test

            # News-line text
            document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
            doc_categories=strip_tags(str(newsline('topics')[0].body))
            doc_categories = unescape(doc_categories)

            document_body = unescape(document_body)

            # News-line categories
            topics = newsline.topics.contents
            places = newsline.places.contents
            people = newsline.people.contents
            orgs = newsline.orgs.contents
            exchanges = newsline.exchanges.contents

            for topic in topics:
                document_categories.append('to_' + strip_tags(str(topic)))

            for place in places:
                document_categories.append('pl_' + strip_tags(str(place)))

            for person in people:
                document_categories.append('pe_' + strip_tags(str(person)))

            for org in orgs:
                document_categories.append('or_' + strip_tags(str(org)))

            for exchange in exchanges:
                document_categories.append('ex_' + strip_tags(str(exchange)))
        #             print "document_categories: ",document_categories
            # Create new document    
        #             update_frequencies(document_categories)

            document_X[document_id] = document_body
            document_Y[document_id] = document_categories


Reading file: reut2-021.sgm


In [60]:
# test_text="Huge oil platforms dot the Gulf like beacons -- usually lit up like Christmas trees at night. One of them, sitting astride the Rostam offshore oilfield,was all but blown out of the water by U.S. Warships on Monday.    The Iranian platform, an unsightly mass of steel andconcrete, was a three-tier structure rising 200 feet (60metres) above the warm waters of the Gulf until four U.S.Destroyers pumped some 1,000 shells into it.    The U.S. Defense Department said just 10 pct of one section of the structure remained.    U.S. helicopters destroyed three Iranian gunboats after an American helicopter came under fire earlier this month and U.S.forces attacked, seized, and sank an Iranian ship they said had been caught laying mines.    But Iran was not deterred, according to U.S. defense officials, who said Iranian forces used Chinese-made Silkworm missiles to hit a U.S.-owned Liberian-flagged ship on Thursday and the Sea Isle City on Friday.    Both ships were hit in the territorial waters of Kuwait, a key backer of Iraq in its war with Iran.    Henry Schuler, a former U.S. diplomat in the Middle Eastnow with CSIS said Washington had agreed to escort Kuwaiti tankers in order to deter Iranian attacks on shipping.    But he said the deterrence policy had failed and the level of violence and threats to shipping had increased as a result of U.S. intervention and Iran's response.    The attack on the oil platform was the latest example of a U.S. \"tit-for-tat\" policy that gave Iran the initiative, said Harlan Ullman, an ex-career naval officer now with CSIS.    He said with this appraoch America would suffer \"the deathof one thousand cuts.\"    But for the United States to grab the initiative litarily, it must take warlike steps such as mining Iran's harbors or blockading the mouth of the Gulf through which itsshipping must pass, Schuler said.    He was among those advocating mining as a means of bringing Iran to the neogtiating table. If vital supplies were cut off,Tehran could not continue the war with Iraq.    Ullman said Washington should join Moscow in a diplomatic initiative to end the war and the superpowers should impose anarms embargo against Tehran if it refused to negotiate.    He said the United States should also threaten to mine and blockade Iran if it continued fighting and must press Iraq to acknowledge responsibility for starting the war as part of asettlement.    Iranian and Western diplomats say Iraq started the war by invading Iran's territory in 1980. Iraq blames Iran for theoutbreak of hostilities, which have entailed World War I-stylenfantry attacks resulting in horrific casualties.    Each side has attacked the others' shipping." 
test_total_X,test_encoded_X=create_x_matrix(document_X)
y=nn.predict(test_encoded_X)

In [61]:
print(y)
# print(document_Y)

[[0.47443122 0.28245524 0.12903568 0.05883632 0.0552416 ]
 [0.9410566  0.03763089 0.00697364 0.00691565 0.00742319]
 [0.47443122 0.28245524 0.12903568 0.05883632 0.0552416 ]
 ...
 [0.6807208  0.11202405 0.0671718  0.10630406 0.03377933]
 [0.91882724 0.0377016  0.01542944 0.02137366 0.00666803]
 [0.8990815  0.07304464 0.00949921 0.01706268 0.00131201]]


In [62]:
output=[]
for i in y:
    dict_temp={'Topics':i[0],'Places':i[1],'Peoples':i[2],'Exchanges':i[3],'Organizations':i[4]}
    output.append(dict_temp)     

In [38]:
# print(output[0])
ranked_output=[]
for i in output:
    t={}
    for key, value in sorted(i.items(), key=lambda item: item[1]):
        t[key]=value
    rank=0
    for k in t.keys():
        t[k]=rank
        rank+=1
    ranked_output.append(t)

In [39]:
print(ranked_output)

[{'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Peoples': 1, 'Organizations': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Peoples': 1, 'Organizations': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Organizations': 0, 'Exchanges': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Organizations': 0, 'Peoples': 1, 'Places': 2, 'Exchanges': 3, 'Topics': 4}, {'Organizations': 0, 'Exchanges': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Organizations': 0, 'Exchanges': 1, 'Peoples': 2, 'Places': 3, 'Topics': 4}, {'Exchanges': 0, 'Organizations': 1, 'Peoples': 2, 'Places': 3,